In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patheffects as path_effects


file_path = r"E:\Projects\Gamezone Orders Data\Data\Cleaned\gamezone_orders_data_cleaned.csv"
df = pd.read_csv(
    file_path,
    parse_dates=['purchase_ts_cleaned', 'ship_ts'],
    dtype={
        'purchase_year': 'Int64',
        'purchase_month': 'Int64',
        'time_to_ship': 'Int64',
        'revenue': 'float'
    },
    encoding='utf-8',
    keep_default_na=False,
    na_values=['']  # Only treat empty strings as NaN
)

In [3]:
# --- 1️⃣ CORE PERFORMANCE ---
sales_kpis = df.groupby('purchase_month').agg(
    total_revenue=('revenue', 'sum'),
    total_orders=('order_id', 'count'),
    unique_customers=('user_id', 'nunique')
).reset_index()
sales_kpis = sales_kpis[(sales_kpis['purchase_month'] > 0) & (sales_kpis['purchase_month'] <= 12)]
sales_kpis['aov'] = sales_kpis['total_revenue'] / sales_kpis['total_orders']
sales_kpis['revenue_growth_%'] = sales_kpis['total_revenue'].pct_change() * 100
sales_kpis['orders_growth_%'] = sales_kpis['total_orders'].pct_change() * 100

# --- OPTION A: True Chronological Cumulative (separate calculation) ---
df_sorted = df.sort_values('purchase_ts_cleaned').copy()
df_sorted['cumulative_revenue_chronological'] = df_sorted['revenue'].cumsum()
# Get the max cumulative revenue for each month for reference
chronological_by_month = df_sorted.groupby('purchase_month')['cumulative_revenue_chronological'].max().reset_index()
chronological_by_month.columns = ['purchase_month', 'cumulative_revenue_chronological']

# --- OPTION B: Average Monthly Pattern (Typical Cumulative) ---
# First need to know how many actual months of data contributed to each aggregated month
months_with_data = df.groupby('purchase_month').agg(
    year_count=('purchase_ts_cleaned', lambda x: x.dt.year.nunique())
).reset_index()
sales_kpis = sales_kpis.merge(months_with_data, on='purchase_month', how='left')
sales_kpis['avg_monthly_revenue'] = sales_kpis['total_revenue'] / sales_kpis['year_count']
sales_kpis['cumulative_revenue_typical'] = sales_kpis['avg_monthly_revenue'].cumsum()

# --- OPTION C: Year-over-Year Cumulative (separate table) ---
# Extract year and create year-month combinations
df['purchase_year'] = df['purchase_ts_cleaned'].dt.year
yearly_revenue = df.groupby(['purchase_year', 'purchase_month'])['revenue'].sum().reset_index()
yearly_revenue = yearly_revenue.sort_values(['purchase_year', 'purchase_month'])
yearly_revenue['cumulative_revenue_yoy'] = yearly_revenue.groupby('purchase_year')['revenue'].cumsum()

# Pivot to show all years side by side (optional, for easier viewing)
yoy_pivot = yearly_revenue.pivot(index='purchase_month', columns='purchase_year', values='cumulative_revenue_yoy')
yoy_pivot.columns = [f'cumulative_revenue_{int(year)}' for year in yoy_pivot.columns]
yoy_pivot = yoy_pivot.reset_index()

# --- 2️⃣ REVENUE SEGMENTATION ---
revenue_by_product = df.groupby(['purchase_month', 'product_name_cleaned'])['revenue'].sum().unstack().fillna(0)
revenue_by_product.columns = [f"revenue_product_{c}" for c in revenue_by_product.columns]

revenue_by_region = df.groupby(['purchase_month', 'region'])['revenue'].sum().unstack().fillna(0)
revenue_by_region.columns = [f"revenue_region_{c}" for c in revenue_by_region.columns]

revenue_by_channel = df.groupby(['purchase_month', 'marketing_channel_cleaned'])['revenue'].sum().unstack().fillna(0)
revenue_by_channel.columns = [f"revenue_channel_{c}" for c in revenue_by_channel.columns]

revenue_by_platform = df.groupby(['purchase_month', 'purchase_platform'])['revenue'].sum().unstack().fillna(0)
revenue_by_platform.columns = [f"revenue_platform_{c}" for c in revenue_by_platform.columns]

# --- 3️⃣ ADVANCED INSIGHTS ---
def calc_topx_share(df, top_n):
    return (
        df.groupby(['purchase_month', 'product_name_cleaned'])['revenue']
        .sum()
        .groupby(level=0)
        .apply(lambda x: x.nlargest(min(top_n, len(x))).sum() / x.sum() * 100)
        .reset_index(name=f'top{top_n}_products_%')
    )

top1_products = calc_topx_share(df, 1)
top3_products = calc_topx_share(df, 3)

avg_shipping_time = (
    df.groupby('purchase_month')['time_to_ship']
    .mean()
    .reset_index(name='avg_time_to_ship_days')
)

invalid_shipping = (
    df[df['ship_ts'] < df['purchase_ts_cleaned']]
    .groupby('purchase_month')['order_id']
    .count()
    .reset_index(name='invalid_shipping_count')
)

# --- 4️⃣ MERGE EVERYTHING ---
sales_kpis = (
    sales_kpis
    .merge(chronological_by_month, on='purchase_month', how='left')  # Option A
    .merge(top1_products, on='purchase_month', how='left')
    .merge(top3_products, on='purchase_month', how='left')
    .merge(avg_shipping_time, on='purchase_month', how='left')
    .merge(invalid_shipping, on='purchase_month', how='left')
    .merge(revenue_by_product, on='purchase_month', how='left')
    .merge(revenue_by_region, on='purchase_month', how='left')
    .merge(revenue_by_channel, on='purchase_month', how='left')
    .merge(revenue_by_platform, on='purchase_month', how='left')
    .merge(yoy_pivot, on='purchase_month', how='left')  # Option C
)


sales_kpis


,purchase_month,total_revenue,total_orders,unique_customers,aov,revenue_growth_%,orders_growth_%,year_count,avg_monthly_revenue,cumulative_revenue_typical,...,revenue_channel_affiliate,revenue_channel_direct,revenue_channel_email,revenue_channel_social media,revenue_channel_unknown,revenue_platform_mobile app,revenue_platform_website,cumulative_revenue_2019,cumulative_revenue_2020,cumulative_revenue_2021
0,1,499725.27,1821,1465,274.423542,NaN,NaN,3,166575.090000,1.665751e+05,...,20981.53,423803.16,47230.02,4921.70,2788.86,15611.29,484113.98,100491.28,109935.63,289298.36
1,2,514735.06,1825,1818,282.046608,3.003608,0.219660,3,171578.353333,3.381534e+05,...,24073.14,409898.17,71789.35,3649.89,5324.51,12108.61,502626.45,180880.38,295374.22,538205.73
2,3,417653.10,1509,1504,276.774751,-18.860569,-17.315068,2,208826.550000,5.469800e+05,...,21741.97,349057.57,40503.66,6013.92,335.98,14406.92,403246.18,296074.36,597833.34,NaN
3,4,472963.91,1723,1712,274.500238,13.243242,14.181577,2,236481.955000,7.834619e+05,...,21056.43,397316.61,46497.87,7402.57,690.43,13618.58,459345.33,417483.19,949388.42,NaN
4,5,468890.58,1727,1725,271.505837,-0.861235,0.232153,2,234445.290000,1.017907e+06,...,9450.17,409505.52,40931.75,6577.75,2425.39,14994.65,453895.93,540860.11,1294902.08,NaN
5,6,433190.52,1617,1596,267.897662,-7.613729,-6.369427,2,216595.260000,1.234502e+06,...,12386.30,374127.37,38072.96,7595.89,1008.00,8588.97,424601.55,649310.63,1619642.08,NaN
6,7,448530.84,1693,1573,264.932569,3.541241,4.700062,2,224265.420000,1.458768e+06,...,13726.46,378115.60,45543.21,6668.47,4477.10,7738.30,440792.54,774404.87,1943078.68,NaN
7,8,525416.75,1797,1654,292.385504,17.141722,6.142942,2,262708.375000,1.721476e+06,...,23152.59,437146.32,53937.77,3814.63,7365.44,9651.59,515765.16,903957.17,2338943.13,NaN
8,9,618444.77,2177,1860,284.081199,17.705568,21.146355,2,309222.385000,2.030699e+06,...,15443.82,523155.57,68559.25,8669.35,2616.78,16666.22,601778.55,1065530.45,2795814.62,NaN
9,10,464867.06,1549,1520,300.107850,-24.832890,-28.847037,2,232433.530000,2.263132e+06,...,13332.93,404597.08,41843.36,3559.69,1534.00,8246.99,456620.07,1187692.77,3138519.36,NaN


In [4]:
yearly_revenue

,purchase_year,purchase_month,revenue,cumulative_revenue_yoy
0,2019.0,1,100491.28,100491.28
1,2019.0,2,80389.10,180880.38
2,2019.0,3,115193.98,296074.36
3,2019.0,4,121408.83,417483.19
4,2019.0,5,123376.92,540860.11
5,2019.0,6,108450.52,649310.63
6,2019.0,7,125094.24,774404.87
7,2019.0,8,129552.30,903957.17
8,2019.0,9,161573.28,1065530.45
9,2019.0,10,122162.32,1187692.77


In [5]:
# exporting this as csv 
sales_kpis.to_csv('sales_kpis.csv', index=False)
yearly_revenue.to_csv('yearly_revenue.csv', index=False)